In [7]:
import os,sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from statsmodels.stats.multitest import multipletests
from pathlib import Path
from tqdm.auto import tqdm
import re
import glob
from scipy import stats

In [21]:
dir_list = pd.read_csv('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/gw-CRISPRa_from_cluster/h5ad_raw_files_info.csv')
lane_cols_for_eff = [c for c in dir_list.columns if c.startswith("lane")]
run = []
for _, row in dir_list[lane_cols_for_eff].iterrows():
    lane_dirs = []
    for col in lane_cols_for_eff:
        v = row[col]
        if pd.notna(v) and str(v).strip() != "":
            lane_dirs.append(f"{v}_{col}")
    if lane_dirs:
        run.append(lane_dirs)

In [11]:
def summarize_exp_stats(lane_path, lane_list):
    dfs = []  
    for lanes in lane_list:
        df = pd.read_csv(glob.glob(os.path.join(lane_path, lanes, "*_guide_count_info.csv"))[0])
        df.set_index('guide_id', inplace= True)
        dfs.append(df)
    num_cols = ['n_cells','sum_guide','sumsq_guide','ntc_cells','sum_ntc','sumsq_ntc']
    
    summary_ = pd.concat([df[num_cols] for df in dfs], axis=0).groupby('guide_id').sum()
    return summary_

In [12]:
def compute_stats(dict_key, dict_val):
    pseudocount = 5e-2
    df = dict_val.copy()

    # Means
    df["guide_mean"] = df["sum_guide"] / df["n_cells"]
    df["ntc_mean"]   = df["sum_ntc"]   / df["ntc_cells"]

    # Variances (safe when n>=2)
    df["guide_var"] = (df["sumsq_guide"] - (df["sum_guide"]**2)/df["n_cells"]) / (df["n_cells"] - 1)
    df["ntc_var"]   = (df["sumsq_ntc"]   - (df["sum_ntc"]**2)/df["ntc_cells"]) / (df["ntc_cells"] - 1)

    df["guide_std"] = np.sqrt(df["guide_var"])
    df["ntc_std"]   = np.sqrt(df["ntc_var"])

    # Handle NaN/0 std
    df.fillna({'guide_std': 0, 'ntc_std': 0}, inplace=True)
    df.loc[:, "guide_std"] = np.where(df["guide_std"] == 0, 0.01, df["guide_std"])
    df.loc[:, "ntc_std"]   = np.where(df["ntc_std"]   == 0, 0.01, df["ntc_std"])

    # Decide direction
    # CRISPRi: expect guide < NTC
    # CRISPRa: expect guide > NTC
    mode = dict_key
    is_crispri = "i" in mode.split("_")[1] 
    is_crispra = "a" in mode.split("_")[1]  

    # Fold-change definition (do once)
    if is_crispri:
        df["fc"] = (df["ntc_mean"] + pseudocount) / (df["guide_mean"] + pseudocount)  # >1 means knockdown
    elif is_crispra:
        df["fc"] = (df["guide_mean"] + pseudocount) / (df["ntc_mean"] + pseudocount)  # >1 means activation
    else:
        df["fc"] = np.nan  # unknown mode

    # Welch t-test per row
    t_stats, p_ones = [], []
    for _, row in df.iterrows():
        t_stat, p_two = stats.ttest_ind_from_stats(
            row["guide_mean"], row["guide_std"], row["n_cells"],
            row["ntc_mean"],   row["ntc_std"],   row["ntc_cells"],
            equal_var=False
        )

        if is_crispri:
            if (t_stat < 0):
                p_one = (p_two / 2) 
            else:
                p_one = 1.0
        elif is_crispra:
            if (t_stat > 0):
                p_one = (p_two / 2) 
            else:
                p_one = 1.0
        else:
            p_one = np.nan

        t_stats.append(t_stat)
        p_ones.append(p_one)

    df["t_statistic"] = t_stats
    df["p_value"] = p_ones

    # FDR
    mask = df["p_value"].notna()
    df["adj_pvals"] = np.nan
    df.loc[mask, "adj_pvals"] = multipletests(df.loc[mask, "p_value"], method="fdr_bh")[1]

    # Cohen's d (pooled SD)
    n1 = df["n_cells"].astype(float)
    n2 = df["ntc_cells"].astype(float)
    sp2 = ((n1 - 1)*df["guide_var"] + (n2 - 1)*df["ntc_var"]) / (n1 + n2 - 2)
    sp  = np.sqrt(sp2)

    df["effect_size"] = np.nan
    m = (n1 >= 2) & (n2 >= 2) & (sp > 0)
    df.loc[m, "effect_size"] = (df.loc[m, "guide_mean"] - df.loc[m, "ntc_mean"]) / sp[m]

    return df

In [5]:
experiment = {}
lane_path = '/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/gw-CRISPRa_from_cluster'
for i in range(len(run)):
    
    summed_ = summarize_exp_stats(lane_path, run[i])
    prefix = run[i][0].rsplit('_lane', 1)[0]
    experiment[prefix] = summed_
experiment = {k: compute_stats(k, v) for k, v in experiment.items()}

/Users/chandrima.modak/ron_perturb_seq_analysis/gw_CRISPRa_Chandrima/branch_code_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
/Users/chandrima.modak/ron_perturb_seq_analysis/gw_CRISPRa_Chandrima/branch_code_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
/Users/chandrima.modak/ron_perturb_seq_analysis/gw_CRISPRa_Chandrima/branch_code_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
/Users/chandrima.modak/ron_perturb_seq_analysis/gw_CRISPRa_Chandrima/branch_code_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
/Users/chandrima.modak/ron_perturb_seq_analysis/gw_CRISPRa_Chandrima/branch_code_env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
/Users/chandrima.modak/ron_per

In [6]:
for k in experiment.keys():
    outdir = os.path.join('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/gw-CRISPRa_from_cluster/qc_stats_welch', k)
    os.makedirs(outdir, exist_ok= True)
    filename = f"{k}_guide_efficency_welch.csv"
    filepath = os.path.join(outdir, filename)
    experiment[k].to_csv(filepath)

# MACS experiment

In [72]:
def compute_stats_ko(df):
    pseudocount = 5e-2

    # Means
    df["guide_mean"] = df["sum_guide"] / df["n_cells"]
    df["ntc_mean"]   = df["sum_ntc"]   / df["ntc_cells"]

    # Variances (safe when n>=2)
    df["guide_var"] = (df["sumsq_guide"] - (df["sum_guide"]**2)/df["n_cells"]) / (df["n_cells"] - 1)
    df["ntc_var"]   = (df["sumsq_ntc"]   - (df["sum_ntc"]**2)/df["ntc_cells"]) / (df["ntc_cells"] - 1)

    df["guide_std"] = np.sqrt(df["guide_var"])
    df["ntc_std"]   = np.sqrt(df["ntc_var"])

    # Handle NaN/0 std
    df.fillna({'guide_std': 0, 'ntc_std': 0}, inplace=True)
    df.loc[:, "guide_std"] = np.where(df["guide_std"] == 0, 0.01, df["guide_std"])
    df.loc[:, "ntc_std"]   = np.where(df["ntc_std"]   == 0, 0.01, df["ntc_std"])

    # Decide direction
    
    # Fold-change definition (do once)
    df["fc"] = (df["ntc_mean"] + pseudocount) / (df["guide_mean"] + pseudocount)  # >1 means knockdown

    # Welch t-test per row
    t_stats, p_ones = [], []
    for _, row in df.iterrows():
        t_stat, p_two = stats.ttest_ind_from_stats(
            row["guide_mean"], row["guide_std"], row["n_cells"],
            row["ntc_mean"],   row["ntc_std"],   row["ntc_cells"],
            equal_var=False
        )
            
        if (t_stat < 0):
            p_one = (p_two / 2) 
        else:
            p_one = 1.0

        t_stats.append(t_stat)
        p_ones.append(p_one)

    df["t_statistic"] = t_stats
    df["p_value"] = p_ones

    # FDR
    mask = df["p_value"].notna()
    df["adj_pvals"] = np.nan
    df.loc[mask, "adj_pvals"] = multipletests(df.loc[mask, "p_value"], method="fdr_bh")[1]

    # Cohen's d (pooled SD)
    n1 = df["n_cells"].astype(float)
    n2 = df["ntc_cells"].astype(float)
    sp2 = ((n1 - 1)*df["guide_var"] + (n2 - 1)*df["ntc_var"]) / (n1 + n2 - 2)
    sp  = np.sqrt(sp2)

    df["effect_size"] = np.nan
    m = (n1 >= 2) & (n2 >= 2) & (sp > 0)
    df.loc[m, "effect_size"] = (df.loc[m, "guide_mean"] - df.loc[m, "ntc_mean"]) / sp[m]

    return df



In [73]:
dir_list = pd.read_csv('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/experiment_info.csv')
lane_cols_for_eff = [c for c in dir_list.columns]
run = []
for _, row in dir_list[lane_cols_for_eff].iterrows():
    for col in lane_cols_for_eff:
       run.append(f'{row[col]}_{col}')

In [74]:
lane_path = '/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/'
ctrl_list = []
lps_list = []
for i in run:
    if 'Ctrl' in i:
        ctrl_list.append(i)
    else:
        lps_list.append(i)

In [75]:
df_ctrl =  summarize_exp_stats(lane_path, ctrl_list)
df_ctrl

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc
guide_id,,,,,,
A4GALT-1,578,17.812806,15.402874,29713,565.299118,477.864540
A4GALT-2,390,13.338955,10.076600,29713,565.299118,477.864540
A4GALT-3,827,23.268857,18.793370,29713,565.299118,477.864540
A4GALT-4,951,19.837996,16.570582,29713,565.299118,477.864540
AAGAB-1,844,369.662735,363.413422,29713,17498.160645,18865.284668
...,...,...,...,...,...,...
ZNF79-4,645,19.992992,13.534191,29713,1144.810577,886.442139
ZNF823-1,750,47.474886,39.372022,29713,2272.092102,1873.482208
ZNF823-2,719,65.517632,47.691523,29713,2272.092102,1873.482208


In [76]:
df_lps = summarize_exp_stats(lane_path, lps_list)
df_lps

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc
guide_id,,,,,,
A4GALT-1,684,129.025307,121.841091,33201,7422.219360,7432.158569
A4GALT-2,418,84.661737,84.817885,33201,7422.219360,7432.158569
A4GALT-3,1060,248.192352,243.806690,33201,7422.219360,7432.158569
A4GALT-4,1135,225.813587,219.349018,33201,7422.219360,7432.158569
AAGAB-1,893,437.408257,450.637596,33201,21169.553223,23723.407227
...,...,...,...,...,...,...
ZNF79-4,818,19.905251,14.135347,33201,1106.498932,866.111176
ZNF823-1,910,33.340771,27.321822,33201,1335.207062,1085.839233
ZNF823-2,905,45.083932,37.499980,33201,1335.207062,1085.839233


In [77]:
path_fdr = '/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/qc_stats'
df_ctrl_fdr = compute_stats_ko(df_ctrl)
# df_ctrl_fdr.to_csv(os.path.join(path_fdr, 'Macs_ctrl_guide_efficiency.csv'))

In [78]:
df_lps_fdr = compute_stats_ko(df_lps)
# df_lps_fdr.to_csv(os.path.join(path_fdr, 'Macs_lps_guide_efficiency.csv'))

In [79]:
df_ctrl

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc,guide_mean,ntc_mean,guide_var,ntc_var,guide_std,ntc_std,fc,t_statistic,p_value,adj_pvals,effect_size
guide_id,,,,,,,,,,,,,,,,,
A4GALT-1,578,17.812806,15.402874,29713,565.299118,477.864540,0.030818,0.019025,0.025743,0.015721,0.160447,0.125384,0.854083,1.756628,1.000000e+00,1.000000e+00,0.093486
A4GALT-2,390,13.338955,10.076600,29713,565.299118,477.864540,0.034202,0.019025,0.024731,0.015721,0.157261,0.125384,0.819754,1.898002,1.000000e+00,1.000000e+00,0.120599
A4GALT-3,827,23.268857,18.793370,29713,565.299118,477.864540,0.028136,0.019025,0.021960,0.015721,0.148188,0.125384,0.883394,1.750770,1.000000e+00,1.000000e+00,0.072279
A4GALT-4,951,19.837996,16.570582,29713,565.299118,477.864540,0.020860,0.019025,0.017007,0.015721,0.130411,0.125384,0.974106,0.427602,1.000000e+00,1.000000e+00,0.014615
AAGAB-1,844,369.662735,363.413422,29713,17498.160645,18865.284668,0.437989,0.588906,0.239033,0.288116,0.488910,0.536765,1.309263,-8.817987,2.964749e-18,1.485114e-17,-0.281823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF79-4,645,19.992992,13.534191,29713,1144.810577,886.442139,0.030997,0.038529,0.020054,0.028350,0.141610,0.168374,1.092992,-1.330560,9.188857e-02,1.436960e-01,-0.044873
ZNF823-1,750,47.474886,39.372022,29713,2272.092102,1873.482208,0.063300,0.076468,0.048554,0.057207,0.220349,0.239180,1.116223,-1.612790,5.359369e-02,8.694444e-02,-0.055158
ZNF823-2,719,65.517632,47.691523,29713,2272.092102,1873.482208,0.091123,0.076468,0.058108,0.057207,0.241055,0.239180,0.896152,1.611130,1.000000e+00,1.000000e+00,0.061262


### Add cell level guide efficiency

In [80]:
df_ctrl = []
df_lps = []
for i in run:
    cell_ko_pattern = glob.glob(os.path.join('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis', i, '*cells_based_guide_efficiency.csv'))[0]
    df = pd.read_csv(cell_ko_pattern)
    if 'Ctrl' in i:
        df_ctrl.append(df)
    else:
        df_lps.append(df)

In [81]:
dfs_ctrl = pd.concat(df_ctrl,axis=0)
dfs_lps = pd.concat(df_lps,axis=0)
dfs_ctrl['signif_ko_95perc'] = dfs_ctrl['ntc_fraction'] < 0.05
dfs_lps['signif_ko_95perc'] = dfs_lps['ntc_fraction'] < 0.05

In [82]:
def ko_cells_percent(df):
    n_kos = df.groupby('guide_id')['signif_ko_95perc'].sum()
    n_tot = df.groupby('guide_id').size()
    df_ko = pd.DataFrame(n_kos/n_tot, columns= ['percent_cells_signi_ko'])
    df_ko
    return df_ko

In [83]:
df_ctrl_ko = ko_cells_percent(dfs_ctrl)
df_lps_ko = ko_cells_percent(dfs_lps)

In [84]:
df_ctrl_fdr =pd.merge(df_ctrl_fdr, df_ctrl_ko, on = 'guide_id', how= 'inner')
df_lps_fdr = pd.merge(df_lps_fdr, df_lps_ko, on = 'guide_id', how= 'inner')

In [85]:
df_ctrl_fdr.to_csv(os.path.join(path_fdr, 'Macs_Ctrl_cell_level_guide_efficiency.csv'))
df_lps_fdr.to_csv(os.path.join(path_fdr, 'Macs_LPS_cell_level_guide_efficiency.csv'))